In [1]:
import pandas as pd
import struct

In [2]:
fname="/Users/jianlin/Documents/Project_STDF_wafermap/STDF_data/main_Lot_1_Wafer_1_Oct_13_09h33m41s_STDF.stdf"

In [20]:
with open(fname,mode='rb') as file:
    row_data = file.read()
#str_row_data=str(row_data)
str_row_data

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf8 in position 10: invalid start byte

# Filter Method

In [4]:
dict_pack = {
    'FAR':[0,10],
    'ATR':[0,20],
    'MIR':[1,10],
    'MRR':[1,20],
    'PCR':[1,30],
    'HBR':[1,40],
    'SBR':[1,50],
    'PMR':[1,60],
    'PGR':[1,62],
    'PLR':[1,63],
    'RDR':[1,70],
    'SDR':[1,80],
    'WIR':[2,10],
    'WRR':[2,20],
    'WCR':[2,30],
    'PIR':[5,10],
    'PRR':[5,20],
    'TSR':[10,30],
    'PTR':[15,10],
    'MPR':[15,15],
    'FTR':[15,20],
    'BPS':[20,10],
    'EPS':[20,20],
    'GDR':[50,10],
    'DTR':[50,30]
}
#dict_pack['PRR'][0]

In [18]:
def struct_packing(rtype,subtype):
    packing = str(struct.pack("BB",rtype,subtype))
    return packing[2:len(packing)-1]

def data_mark(string_row_data,key_header,key):
    key_list = string_row_data.split(key_header)    #will cut key_header

    for n in range(0,len(dict_position[key])):
        if dict_position[key][n][1]==1:            
            key_list[n] = key_list[n][:len(key_list[n])-8] + "|" + key_list[n][len(key_list[n])-8:]
            key_list[n] = key_list[n]+"%"+key+":"+key_header
        else:
            key_list[n] = key_list[n]+key_header
    
    return "%".join(key_list)


def debug_by_positionrule_2():
    #/need execute before data_marking/
    #position locate
    dict_position = {}

    for key in dict_pack.keys():
        header = struct_packing(dict_pack[key][0],dict_pack[key][1])
        print(str_row_data)
        group_list = str_row_data.split(header)
        group_count = 0
        dict_position[key]=[]
        for num in range(0, len(group_list)-1):
            group_count = group_count + len(group_list[num])
            add_group = [group_count,1]
            dict_position[key].append(add_group)
    
    #ATR:bewteen FAR,MIR
    if dict_position['ATR'][0][0] < dict_position['ATR'][1][0]:
        dict_position['ATR'][1][1]=0
    #MRR:need last
    elif dict_position['MRR'][0][0] < dict_position['MRR'][1][0]:
        dict_position['MRR'][0][1]=0
    #SBR:after HBR
    for l in dict_position['SBR']:
        if l[0] < dict_position['HBR'][0][0]:
            l[1]=0
    #PIR:after PRR
    for l in dict_position['PIR']:
        if l[0] < dict_position['PRR'][0][0]:
            l[1]=0
    
    return dict_position


def extract_data(processed_data):
    record_type_split = processed_data.split("|")
    for data in range(1,len(record_type_split)):
        data_extracted = record_type_split[data].split("%")
    return data_extracted



# execute

In [19]:
# read binary data
with open(fname,mode='rb') as file:
    row_data = file.read()
#str_row_data=str(row_data)
str_row_data=row_data

dict_position = debug_by_positionrule_2()

# data mark
processed_data = str_row_data
for num_key in range(0,25):
    key = list(dict_pack.keys())[num_key]
    header = struct_packing(dict_pack[key][0],dict_pack[key][1])
    processed_data = data_mark(processed_data,header,key)
processed_data

b'\x02\x00\x00\n\x02\x04y\x00\x00\x14\xf8T\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15\x82\x00\x01\n\xeaT\xffW\xf8T\xffW\x01 N \xff\xff \x011\x00\x15localhost.localdomain\t93000-SOC\x05cp_32\x00\x00\x07vincent\x0593000 s/w rev. 7.1.4.3 (E), 15-Oct-12\n\x00\x0525.25\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x01P\x01\x02\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x12\x00\x01<\x01\x00\x01\x00\x03Xin\x0510101\x011\x01\x01\x13\x00\x01<\x02\x00\x02\x00\x04Xout\x0510102\x012\x01\x01\x0e\x00\x01<\x03\x00\x00\x00\x03VCC\x0211\x00\x01\x01\x14\x00\x02\x1e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x80\x00\x80  \x07\x00\x02\n\x01\x02\xf8T\xffW\x00\x02\x00\x05\n\x01\x01\x12\x00\x01<\x01\x00\x01\x00\x03Xin\x0510101\x011\x01\x01\x13\x00\x01<\x02\x00\x02\x00\x04Xout\x0510102\x012\x01\x01\x0e\x00\x01<\x03\x00\x00\x00\x03VCC\x0211\x00\x01\x01\x1a\

TypeError: a bytes-like object is required, not 'str'

In [7]:
#check data integrity
record_type_split = processed_data.split("|")
for data in range(1,len(record_type_split)):
    data_extracted = record_type_split[data].split("%")
    print(len(data_extracted),data_extracted)

3 ['\\x02\\x00', 'FAR:\\x00\\n', '\\x02\\']
3 ['x04y\\x00', 'ATR:\\x00\\x14', '\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15']
3 ['\\x82\\x00', 'MIR:\\x01\\n', '\\xeaT\\xffW\\xf8T\\xffW\\x01 N \\xff\\xff \\x011\\x00\\x15localhost.localdomain\\t93000-SOC\\x05cp_32\\x00\\x00\\x07vincent\\x0593000 s/w rev. 7.1.4.3 (E), ']
3 ['15-Oct-1', 'GDR:2\\n', '\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\']
4 ['x14', '\\x00', 'SDR:\\x01P', '\\x01\\x02\\x01\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00']
3 ['\\x12\\x00', 'PMR:\\x01<', '\\x01\\x00\\x01\\x00\\x03Xin\\x0510101\\x011\\x01\\x01']
4 ['\\x13\\x00', 'PMR:\\x01<', '\\x02\\x00\\x02\\x00\\x04Xout\\x0510102\\x012', '\\x01\\x01']
3 ['\\x0e\\x00', 'PMR:\\x01<', '\\x03\\x00\\x00\\x00\\x03VCC\\x0211']
2 ['\\x00\\x01', 'MRR:\\x01\\']
2 ['x14', '\\x00']
5 ['', 'WCR:\\x0

In [8]:
head_part = []
record_type = []
body_part = []
missing_data = []

record_type_split = processed_data.split("|")
for data in range(1,len(record_type_split)):
    data_extracted = record_type_split[data].split("%")
        
    if len(data_extracted) == 3:
        head_part.append(data_extracted[0])
        record_type.append(data_extracted[1].split(":")[0])
        body_part.append(data_extracted[2])
    else:
        missing_data.append(data_extracted)       
        
print("h",head_part)
print("\nr",record_type)
print("\nb",body_part)
print("\nm",missing_data)


h ['\\x02\\x00', 'x04y\\x00', '\\x82\\x00', '15-Oct-1', '\\x12\\x00', '\\x0e\\x00', '\\x12\\x00', '\\x0e\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1a\\x00', '\\x02\\x00', '\\x1b\\x00', '\\x02\\x00', '\\x1b\\x00', '\\x02\\x00', '\\x1b\\x00', '\\x02\\x00', '\\x1b\\x00', 'x00 \\x00', '\\x0e\\x00', '\\x0e\\x00', '\\x0e\\x00', '\\x0e\\x00', '\\x16\\x00', '\\x16\\x00', '\\x07\\x00']

r ['FAR', 'ATR', 'MIR', 'GDR', 'PMR', 'PMR', 'PMR', 'PMR', 'PRR', 'PIR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'PIR', 'PRR', 'WRR', 'HBR', 'HBR', 'SBR', 'SBR', 'PCR', 'PCR', 'MRR']

b ['\\x02\\', '\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15', '\\xeaT\\

In [9]:
zip_data = zip(head_part,record_type,body_part)
list_data = list(zip_data)

In [10]:
df = pd.DataFrame(list_data,columns=['rec_length','rec_type','rec_body'])
df

,rec_length,rec_type,rec_body
0,\x02\x00,FAR,\x02\
1,x04y\x00,ATR,\xf8T\xffWtdataformatter tester config; Ver: S...
2,\x82\x00,MIR,\xeaT\xffW\xf8T\xffW\x01 N \xff\xff \x011\x00\...
3,15-Oct-1,GDR,\x00\x0525.25\x00\x00\x00\x00\x00\x00\x00\x00\...
4,\x12\x00,PMR,\x01\x00\x01\x00\x03Xin\x0510101\x011\x01\x01
5,\x0e\x00,PMR,\x03\x00\x00\x00\x03VCC\x0211
6,\x12\x00,PMR,\x01\x00\x01\x00\x03Xin\x0510101\x011\x01\x01
7,\x0e\x00,PMR,\x03\x00\x00\x00\x03VCC\x0211\x00\x01\x01
8,\x1a\x00,PRR,\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfb\xff\x...
9,\x02\x00,PIR,\x01\x01


In [11]:
df.loc[8].rec_body

'\\x01\\x01\\x00\\x00\\x00\\x01\\x00\\x01\\x00\\xfb\\xff\\xfc\\xff\\x03\\x02\\x00\\x00\\x011\\x050.000\\x00'

In [12]:
df[df.rec_type=='PRR'].iloc[0]

rec_length                                             \x1a\x00
rec_type                                                    PRR
rec_body      \x01\x01\x00\x00\x00\x01\x00\x01\x00\xfb\xff\x...
Name: 8, dtype: object

In [13]:
prr = ['U1','U1','B1','U2','U2','U2','I2','I2','U4','Cn','Cn','Bn']
fmt = "BBBHHHhhL"

In [14]:
1|1|0|0|1|1|-5|-4|515|1|0.000|[]\n
1,1,0,0,1,1,-5,-4,515,1,0.000,0


SyntaxError: unexpected character after line continuation character (3322058373.py, line 1)

In [26]:
# header of FAR
enc = struct.pack("<BBBHHHhhLx", 1,1,0,0,1,1,-5,-4,515,)
print(enc)
str(enc)

b'\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfb\xff\xfc\xff\x03\x02\x00\x00\x00'


"b'\\x01\\x01\\x00\\x00\\x00\\x01\\x00\\x01\\x00\\xfb\\xff\\xfc\\xff\\x03\\x02\\x00\\x00\\x00'"

In [ ]:
data = "\x0C\x00Maximilianus\xf4\x01".encode('latin1')
number = struct.unpack('<h', data[:2])[0]
print('number:', number)
data = data[2:]
print('text:', data[:number].decode())

In [ ]:
df.loc[8].rec_body

In [ ]:
df.loc[8].rec_body.encode('latin1')

In [29]:
import pyperclip
s1 = print(enc)
pyperclip.copy(s1)

b'\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfb\xff\xfc\xff\x03\x02\x00\x00\x00'


PyperclipException: only str, int, float, and bool values can be copied to the clipboard, not NoneType